In [1]:
from dotenv import load_dotenv
load_dotenv(r'D:\AI ML\Gen AI\Lang_Chain\.env')

True

In [2]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
llm = HuggingFaceEndpoint(
    repo_id='google/gemma-2-2b-it',
    task='conversational'
    )
model = ChatHuggingFace(llm=llm)


In [3]:
# Data Ingestion : From the website we need to scrapte the data
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader('https://en.wikipedia.org/wiki/Generative_artificial_intelligence')

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
docs =loader.load()

In [5]:
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Generative_artificial_intelligence', 'title': 'Generative artificial intelligence - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nGenerative artificial intelligence - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nHistory\n\

In [6]:
## Load Data -> Docs -> Divide our text into chunks --> text --> vectors --> Vector Embeddings --> Vector Store


from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

docs = text_splitter.split_documents(docs)

In [7]:
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Generative_artificial_intelligence', 'title': 'Generative artificial intelligence - Wikipedia', 'language': 'en'}, page_content='Generative artificial intelligence - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nHistory\n\n\n\n\nT

In [8]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model='sentence-transformers/all-MiniLM-L6-v2')

In [9]:
from langchain_community.vectorstores import FAISS

vectordb = FAISS.from_documents(docs,embedding)

In [10]:
vectordb

In [11]:
query = 'Application of Generative AI'
result = vectordb.similarity_search(query)

In [12]:
from IPython.display import Markdown
Markdown(result[0].page_content)

Glossary
Glossary
vte
Generative artificial intelligence (Generative AI, or GenAI[1]) is a subfield of artificial intelligence that utilizes generative models to generate text, images, videos, audio, software code or other forms of data.[2][3] These models learn the underlying patterns and structures of their training data and use them to produce new data[4][5] in response to input, which often comes in the form of natural language prompts.[6][7]

In [13]:
## Retrieval Chain, Document Chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [14]:
prompt = ChatPromptTemplate.from_template(
    '''
Answer the following question based only on the provided context.
{context}
</context>
'''

)

In [17]:
from langchain_core.documents import Document

In [15]:
document_chain = create_stuff_documents_chain(model,prompt)

In [20]:
retriever = vectordb.as_retriever()


query = 'Application of Generative AI'
response =document_chain.invoke({
    'input' : query,
    'context': retriever.invoke(query)
})


Markdown(response)

The provided text describes what Generative AI is and how it works, stating that it can:

* Learn patterns from data.
* Generate new data (like text, images, etc.) in response to prompts. 
* Be multimodal and generate across different type of outputs.


The text also mentions some specific subtypes or models of generative AI: 

* GPTs (Generative Pre-Trained Transformers)
* GANs (Generative Adversarial Networks)
* VAEs (Variational Autoencoders)


Finally, it touches upon some modalities used within Generative AI, such as:

* "Generative AI pornography": AI generating explicit material.
* "Procedural generation": Creating data under algorithmic methods.

Let me know if you'd like more details! 
